In [ ]:
import numpy as np

import sys
sys.path.append('../pysuperres')

%matplotlib notebook
import matplotlib.pyplot as plt

from platform import python_version

print('Python Version: ', python_version())
print('This Notebook has been tested on python 3.6.9.')


%load_ext autoreload
%autoreload 2

In [ ]:
from noise_model import matlab_possion_noise

a = np.array([[1,2,3,54],[2,3,5,10]])
b = 10**12*matlab_possion_noise(a*10**-12,clip=False,seed=None)
print(b)

#Example with a Gaussian
x = np.arange(-10,10+0.01,0.02)
delta = 0.5
num_photons_plot = 1000
Int_plot = (np.exp(-(x-delta/2)**2/2) + np.exp(-(x+delta/2)**2/2))
Int_plot = num_photons_plot * Int_plot / sum(Int_plot)
Int_plot_noise = 10**12*matlab_possion_noise(Int_plot*10**-12, clip=True)


plt.figure()
plt.plot(x,Int_plot_noise)
plt.plot(x,Int_plot)


In [ ]:
from pysuperres import PYSUPERRES_1D, sinc

results_as1d_c = []

thetas = np.arange(0.05,1.2,0.05) * np.pi
print(thetas.shape)

N_exp = 400

np.random.seed(680)

for seed_i in range(N_exp):
    if seed_i%10==0:
        print(seed_i, ' of ', N_exp)
    
    y_gt = []

    for theta in thetas:
        delay1 = theta/2
        delay2 = -theta/2
        
        Photon_number = 1000

        y1 = sinc(x,delay=delay1)**2
        y2 = sinc(x,delay=delay2)**2
        
        y1_gt = y1+y2
        z_max = np.max(0.5*y1_gt)
        y1_gt = Photon_number * y1_gt / np.sum(y1_gt)
        y_max = np.max(y1_gt)
        y1_gt = 10**12*matlab_possion_noise(y1_gt*10**-12)
        y1_gt = y1_gt * z_max / y_max #scaling so that amplitude of the intensities need not be estimated

        y_gt.append(y1_gt)
    
    results_as1d = []

    for i in range(len(thetas)):
        model = PYSUPERRES_1D(x=x, 
                           n_components=2, 
                              #Can be 3 or 4 components for beyond 2 point super-resolution 
                           psf = 'sinc',
                           variables_to_opt = [1, 0, 0], #[location, amplitude, scale]
                              #While Location and Amplitude estimation in the wild works quite well,
                              #scale estimation is still experimental
                           optimizer = 'adam',
                           max_iterations = 1000,
                           init_learning_rate = 0.001,
                           seed=123) #seed used for same initialization for all experiment
        
        y_sig = y_gt[i] 
        model.fit(y_sig, verbose=False)
        estimate = model.est
        delta_est = np.abs(estimate[0,0]-estimate[1,0])
        results_as1d.append(delta_est)

    results_as1d = np.array(results_as1d)  
    results_as1d_c.append(results_as1d)

    
results_as1d_c = np.array(results_as1d_c)

In [ ]:
results_as1d_c = np.array(results_as1d_c)
res1 = results_as1d_c[:,:10]
th = thetas[:10]/np.pi

print(np.sum(res1/np.pi<0.001, axis=0))

plt.figure()
for i in range(len(th)):
    z = res1[:,i].reshape(-1)/np.pi
    plt.scatter([th[i]]*len(z), z, s=0.1)
plt.plot(th, th)

In [ ]:
results_as1d_mean = np.mean(results_as1d_c/np.pi, axis=0)
results_as1d_std = np.std(results_as1d_c/np.pi, axis=0)

print('Estimates: ', results_as1d_mean)
print('Standard Deviations: ', results_as1d_std)

plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.plot(thetas[:10]/np.pi, thetas[:10]/np.pi, 'o-', label='Ground Truth')
plt.errorbar(thetas[:10]/np.pi, results_as1d_mean[:10], results_as1d_std[:10], fmt='o', label='Non-linear with ADAM')

plt.grid('on')
plt.xlim([-0.02,0.52])
plt.xlabel('Ground Truth $\\theta/\pi$')
plt.ylabel('Estimated $\\theta/\pi$')

plt.legend()

results_as1d_var = np.var(results_as1d_c/np.pi, axis=0)
as1d_FI = 1/(results_as1d_var*Photon_number)

plt.subplot(1,3,2)
plt.title('Variance')
plt.xlim([-0.02,0.52])
plt.plot(thetas/np.pi, results_as1d_var*Photon_number, 'o')

plt.subplot(1,3,3)

plt.title('Fisher Information')
plt.plot(thetas/np.pi, as1d_FI, 'o')
plt.xlim([-0.02,0.52])